In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../../source')

import numpy as np
import keras
from keras import backend
import tensorflow as tf
from tensorflow.python.platform import flags

from cleverhans.utils_mnist import data_mnist
from cleverhans.utils_tf import model_train, model_eval
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils import AccuracyReport
from cleverhans.utils_keras import cnn_model
from cleverhans.utils_keras import KerasModelWrapper

from data_processing import import_images_from_directories

/home/josh/.local/share/virtualenvs/assembly_melt-zSdd0Kve/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


NOTE THIS NOTEBOOK USES THE TRAINING DATA AS THE TESTING DATA. TBD on fixing this. I think the overall takeaway (that fgsm works vs our whitebox trained model) still holds.

In [3]:
# dataset_path = '../../data/cifar-11'
dataset_path = '../../data/private_proof_v1'

In [4]:
# desired_image_size = (28, 28)
desired_image_size = (250, 250)
max_images_per_class = 1000

X_train, Y_train = import_images_from_directories(
    dataset_path, 
    resize_shape=desired_image_size, 
    max_images_per_class=max_images_per_class
)

num_classes = Y_train.shape[1]

print("X_train, Y_train shape: {}, {}".format(X_train.shape, Y_train.shape))

X_test, Y_test = X_train, Y_train

Unable to open file ../../data/private_proof_v1/font/font_103.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_142.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_150.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_156.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_169.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_175.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_176.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_200.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_223.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_234.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_243.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_249.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font

/home/josh/.local/share/virtualenvs/assembly_melt-zSdd0Kve/lib/python3.5/site-packages/PIL/Image.py:916: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


Unable to open file ../../data/private_proof_v1/font/font_336.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_337.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_344.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_364.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_409.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_411.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_414.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_432.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_435.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_439.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_440.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font/font_461.jpg, skipping...
Unable to open file ../../data/private_proof_v1/font

Unable to open file ../../data/private_proof_v1/ring/ring_626.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_632.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_64.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_646.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_653.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_654.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_666.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_683.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_687.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_691.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_702.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/ring_708.jpg, skipping...
Unable to open file ../../data/private_proof_v1/ring/

/home/josh/.local/share/virtualenvs/assembly_melt-zSdd0Kve/lib/python3.5/site-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


Unable to open file ../../data/private_proof_v1/insect/insect_75.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_162.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_355.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_428.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_482.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_498.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_549.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_55.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_560.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_577.jpg, skipping...
Unable to open file ../../data/private_proof_v1/machinery/machinery_579.jpg, skipping...
Unable to open file ../../dat

Unable to open file ../../data/private_proof_v1/architecture/architecture_66.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_672.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_673.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_685.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_697.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_705.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_706.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_71.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_713.jpg, skipping...
Unable to open file ../../data/private_proof_v1/architecture/architecture_73.jpg, skipping...
Unable to open file ../../data/private_proof_v1/archi

In [5]:
# Define evaluation functions to make life a bit easier downstream

def evaluate():
    # Evaluate the accuracy of the MNIST model on legitimate test examples
    eval_params = {'batch_size': batch_size}
    acc = model_eval(sess, x, y, preds, X_test, Y_test, args=eval_params)
    report.clean_train_clean_eval = acc
#     assert X_test.shape[0] == test_end - test_start, X_test.shape
    print('Test accuracy on legitimate examples: %0.4f' % acc)
    
def evaluate_2():
    # Accuracy of adversarially trained model on legitimate test inputs
    eval_params = {'batch_size': batch_size}
    accuracy = model_eval(sess, x, y, preds_2, X_test, Y_test,
                          args=eval_params)
    print('Test accuracy on legitimate examples: %0.4f' % accuracy)
    report.adv_train_clean_eval = accuracy

    # Accuracy of the adversarially trained model on adversarial examples
    accuracy = model_eval(sess, x, y, preds_2_adv, X_test,
                          Y_test, args=eval_params)
    print('Test accuracy on adversarial examples: %0.4f' % accuracy)
    report.adv_train_adv_eval = accuracy

In [6]:
# Train the whitebox model
# General setup

n_epochs = 20
batch_size = 128
learning_rate = 0.001

# Object used to keep track of (and return) key accuracies
report = AccuracyReport()
    
# Create TF session and set as Keras backend session
sess = tf.Session()

# Set TF random seed to improve reproducibility
tf.set_random_seed(1234)
rng = np.random.RandomState([2017, 8, 30])

keras.backend.set_session(sess)

if keras.backend.image_dim_ordering() != 'tf':
    keras.backend.set_image_dim_ordering('tf')
    print("INFO: '~/.keras/keras.json' sets 'image_dim_ordering' to "
          "'th', temporarily setting to 'tf'")
    
keras.layers.core.K.set_learning_phase(0)

# Define input TF placeholder
x = tf.placeholder(tf.float32, shape=(None, desired_image_size[0], desired_image_size[1], 3))
y = tf.placeholder(tf.float32, shape=(None, num_classes))

# Define TF model graph
model = cnn_model(img_rows=desired_image_size[0], 
                  img_cols=desired_image_size[1], 
                  channels=3, 
                  nb_filters=64, 
                  nb_classes=num_classes)
preds = model(x)
print("Defined TensorFlow model graph.")


# Train an MNIST model
train_params = {
    'nb_epochs': n_epochs,
    'batch_size': batch_size,
    'learning_rate': learning_rate
}

model_train(sess, x, y, preds, X_train, Y_train, evaluate=evaluate,
            args=train_params, rng=rng)

Defined TensorFlow model graph.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Test accuracy on legitimate examples: 0.1321
Test accuracy on legitimate examples: 0.1575
Test accuracy on legitimate examples: 0.1743
Test accuracy on legitimate examples: 0.2049
Test accuracy on legitimate examples: 0.2410
Test accuracy on legitimate examples: 0.2616
Test accuracy on legitimate examples: 0.2654
Test accuracy on legitimate examples: 0.2901
Test accuracy on legitimate examples: 0.3222
Test accuracy on legitimate examples: 0.3227
Test accuracy on legitimate examples: 0.3603
Test accuracy on legitimate examples: 0.3817
Test accuracy on legitimate examples: 0.3958
Test accuracy on legitimate examples: 0.4043
Test accuracy on legitimate examples: 0.4292
Test accuracy on legitimate examples: 0.4434
Test accuracy on legitimate examples: 0.4576
Test accuracy 

True

In [7]:
# Wrap the FGSM attack object around the whitebox model and evaluate it on the test data

# Initialize the Fast Gradient Sign Method (FGSM) attack object and graph
wrap = KerasModelWrapper(model)
fgsm = FastGradientMethod(wrap, sess=sess)
fgsm_params = {'eps': 0.3,
               'clip_min': 0.,
               'clip_max': 1.}
adv_x = fgsm.generate(x, **fgsm_params)
# Consider the attack to be constant
adv_x = tf.stop_gradient(adv_x)
preds_adv = model(adv_x)

# Evaluate the accuracy of the MNIST model on adversarial examples
eval_par = {'batch_size': batch_size}
acc = model_eval(sess, x, y, preds_adv, X_test, Y_test, args=eval_par)
print('Test accuracy on adversarial examples: %0.4f\n' % acc)
report.clean_train_adv_eval = acc

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Test accuracy on adversarial examples: 0.1317

